In [ ]:
# create simple train test split and calc errors
# model 1: current random forest
# model 2: all features
# model 3: something in the middle
# create split across trees (with groups)
# create split across time
# create split across trees and time
# check leakage across time and groups
# benchmarks?
# with sensor and without as feature
# differen horizon
# Questions:
# Ein Model je Tiefe oder Tiefe als Variable?
# Ein 

%load_ext autoreload
%autoreload 2
import pandas as pd
from sqlalchemy import create_engine
from sklearn.ensemble import RandomForestRegressor
import pickle
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error


import os

import sys
sys.path.append("..")
sys.path.append("../../../qtrees-ai-data-private")

from qtreesprivate.preprocessed_data import Preprocessor

from qtrees.helper import get_logger, init_db_args
from qtrees.constants import NOWCAST_FEATURES, FORECAST_FEATURES

pd.set_option('display.max_columns', None)
DEPTH_MAP = {1: "30 cm", 2: "60 cm", 3: "90 cm"}

CAT_FEATURES = ["baumscheibe_surface", "gattung", "month", "baumscheibe_cat", "standalter_cat"]
NUM_FEATURES = ["water_sga","water_gdk", "shading_index", "wind_max_ms", "rainfall_mm", "temp_avg_c", "ghi_sum_whm2", "gestern", "upm"]

In [ ]:
# dev
#user = "qtrees_user"
#postgres_passwd = ""
#db_qtrees = "qtreesdev-iac-rds.ct3edyn2bzjb.eu-central-1.rds.amazonaws.com"
#
#engine = create_engine(
#    f"postgresql://{user}:{postgres_passwd}@{db_qtrees}:5432/qtrees"
#)
#
#pp = Preprocessor(engine)
#pp.processing_pipeline()
#data = pp.data
#data.info()
#data.to_pickle("data.p")

#tree_devices = pd.read_sql(con= engine, sql="SELECT * FROM private.tree_devices")
#tree_devices.to_pickle("tree_devices.p")

In [ ]:
data = pd.read_pickle("data.p")
data = data.drop("gestern", axis=1)
temp = data.groupby(["timestamp", "type_id"])["value"].mean().shift(1).rename("gestern")
data = data.merge(temp, left_on=["timestamp", "type_id"], right_index=True)

tree_devices = pd.read_pickle("tree_devices.p")

In [ ]:
data

In [ ]:
FEATURES_POC = ["type_id", "temp_avg_c", "rainfall_mm", "water_sga", "water_gdk", "month", "gestern"] #, "standalter_cat", "shading_index", 
FEATURES_MVP = ["type_id", "shading_index", "standalter_cat", "temp_avg_c", "rainfall_mm", "water_sga", "water_gdk", "month", "gestern", "baumscheibe_surface", "gattung", "baumscheibe_cat", "wind_max_ms", "ghi_sum_whm2", "upm"]
FEATURE_NAIVE = ["gestern"]

clean_idx = data.loc[data.timestamp < "2023-09", FEATURES_MVP].dropna().index
clean_data = data.loc[clean_idx]
clean_data = clean_data.merge(tree_devices[["tree_id", "site_id"]], how="left", left_on="tree_id", right_on="tree_id")

#create categories
CAT_FEATURES = ["baumscheibe_surface", "gattung", "month", "baumscheibe_cat", "standalter_cat"]
for c in [x for x in FEATURES_MVP if x in CAT_FEATURES]:
    clean_data.loc[:,c] = clean_data.loc[:,c].factorize()[0]

for c in [x for x in FEATURES_MVP if x in NUM_FEATURES]:
    clean_data.loc[:,c] = (clean_data.loc[:,c]-clean_data.loc[:,c].mean())/clean_data.loc[:,c].std()

clean_data.shape

In [ ]:
clean_data

In [ ]:
clean_data.groupby(pd.Grouper(key='timestamp', freq='M')).size()

In [ ]:
np.random.seed(42)
site_ids = clean_data.site_id.unique()
test_site_ids = np.random.choice(site_ids, int(.25*len(site_ids)), replace=False) 
#test_site_ids = [316,  68, 235,  55,  18,  62] #,  74,  65, 321,  69,  95]
test_site_ids

In [ ]:
def pick_test_weeks(weeks, n_train_weeks=5, n_test_weeks=4):
    train_weeks, test_weeks = [], []

    i = 0
    while i + n_train_weeks  < len(weeks):
        # Define the training period
        train_start = weeks[i]
        train_end = weeks[i + min(n_train_weeks, len(weeks) - i) - 1]

        # Define the testing period
        test_start = weeks[i + n_train_weeks]
        if i + n_train_weeks + n_test_weeks < len(weeks):
            test_end = weeks[i + n_train_weeks + n_test_weeks -1]
        else:
            test_end = weeks[-1]
        
        train_weeks.extend(range(train_start, train_end + 1))  
        test_weeks.extend(range(test_start, test_end + 1)) 

        i = i + n_train_weeks + n_test_weeks 

    return train_weeks, test_weeks
    
weeks = sorted(clean_data.timestamp.dt.isocalendar().week.unique())
train_weeks, test_weeks = pick_test_weeks(weeks)
train_weeks, test_weeks 

In [ ]:
train_mask = ~clean_data.site_id.isin(test_site_ids)
train_data = clean_data[train_mask]

#test_mask = (clean_data.site_id.isin(test_site_ids)) & (clean_data.timestamp.dt.isocalendar().week.isin(test_weeks))
test_mask = (clean_data.site_id.isin(test_site_ids))
test_data = clean_data[test_mask]

print(train_data.shape, test_data.shape)

In [ ]:
y_test

In [ ]:
X_train, y_train = train_data[FEATURES_MVP], train_data["value"]

d = 4
rf = RandomForestRegressor(max_features="sqrt", n_estimators=1000, max_depth=d, bootstrap=True)
rf.fit(X_train, y_train)

# Testen je Tiefe
def predict_depth(model, test_data, depth = 1):
    X_test, y_test = test_data.loc[test_data.type_id==depth, FEATURES_MVP], test_data.loc[test_data.type_id==depth, "value"]
    
    y_hat = model.predict(X_test)
    rmse = mean_squared_error(y_test, y_hat, squared=False)
    mae = mean_absolute_error(y_test, y_hat)
    print(X_test.columns)
    print(model.feature_importances_)
    print(f"Random Forest: Tiefe {DEPTH_MAP[depth]}: RMSE {rmse:.2f}, MAE {mae:.2f}")


    y_hat_benchmark = test_data.loc[test_data.type_id==depth, "gestern"]
    rmse = mean_squared_error(y_test, y_hat_benchmark, squared=False)
    mae = mean_absolute_error(y_test, y_hat_benchmark)
    print(f"Benchmark: Tiefe {DEPTH_MAP[depth]}: RMSE {rmse:.2f}, MAE {mae:.2f} \n")

predict_depth(rf, test_data, depth = 1)
predict_depth(rf, test_data, depth = 2)
predict_depth(rf, test_data, depth = 3)

In [ ]:
X_train, y_train = train_data[FEATURES_MVP], train_data["value"]
X_test, y_test = test_data[FEATURES_MVP], test_data["value"]

for d in [1, 2,3, 4, 8, 12, 16]:
    rf = RandomForestRegressor(max_features="sqrt", n_estimators=1000, max_depth=d, bootstrap=True)
    rf.fit(X_train, y_train)
    y_hat = rf.predict(X_test)



    rmse = mean_squared_error(y_test, y_hat, squared=False)
    mae = mean_absolute_error(y_test, y_hat)

    print(f"Tiefe {d}: RMSE {rmse}, MAE {mae}")

In [ ]:
rf.feature_importances_

In [ ]:
X_train.columns

In [ ]:
residuals.mean()

In [ ]:
residuals = y_test - y_hat

plt.hist(residuals)

In [ ]:
residuals.mean()

In [ ]:
X_train, y_train = train_data[FEATURES_MVP], train_data["value"]
X_test, y_test = test_data[FEATURES_MVP], test_data["value"]
rf = RandomForestRegressor()
rf.fit(X_train, y_train)
y_hat = rf.predict(X_test)

rmse = mean_squared_error(y_test, y_hat, squared=False)
mae = mean_absolute_error(y_test, y_hat)

print(mae, rmse)

In [ ]:
y_test = test_data["value"]

rmse = mean_squared_error(y_test, test_data[FEATURE_NAIVE], squared=False)
mae = mean_absolute_error( y_test, test_data[FEATURE_NAIVE])
print(mae, rmse)